# **NEMO**


In a clean directory we will install NEMO, with the PGPLOT graphics device driver. We will create and integrate a Plummer sphere, and manually compile a NEMO program.

This first cell will re-install a clean version of the source code, from github. Since the code is open source, you will not need an account on github, but the **git** command needs to be present on your system.



In [1]:
cd $HOME
mkdir -p NEMO
cd NEMO
rm -rf demo
git clone https://github.com/teuben/nemo demo

Cloning into 'demo'...
remote: Enumerating objects: 85791, done.
remote: Counting objects: 100% (2170/2170), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 85791 (delta 2110), reused 2091 (delta 2051), pack-reused 83621 (from 4)
Receiving objects: 100% (85791/85791), 48.59 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (61507/61507), done.


The next cell can always be revisited to redo a configure and build. But we show the version of NEMO just to make sure we are in the right directory

In [2]:
cd $HOME/NEMO/demo
git rev-list --count HEAD
cat VERSION

12318
4.5.1


Now we can configure the package.  We need to configure with a YAPP graphics driver.  YAPP stands for *Yet Another Plotting Package*.  If "pgplot" does not work, pick the "ps" (postscript) driver.   There is also an "svg" in development which might be more fun to display actual results, though in practice the PGPLOT
driver is normally used.

In [3]:
date0=$(date)

./configure --with-yapp=pgplot


checking NEMO config... checking build system type... x86_64-unknown-linux-gnu
checking host system type... x86_64-unknown-linux-gnu
Found CANONICAL_HOST = linux-gnu or x86_64-unknown-linux-gnu or x86_64
Found CANONICAL_BUILD = x86_64-unknown-linux-gnu
checking target system type... x86_64-unknown-linux-gnu
Found CANONICAL_TARGET = x86_64-unknown-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C... yes
checking whether gcc accepts -g... yes
checking for gcc option to enable C11 features... none needed
checking for stdio.h... yes
checking for stdlib.h... yes
checking for string.h... yes
checking for inttypes.h... yes
checking for stdint.h... yes
checking for strings.h... yes
checking for sys/stat.h... yes
checking for sys/types.h... 

The next step is building the software (libraries and binaries) in place. NEMO does normally not install itself in some /usr/local/bin type directory, the build is done in-place. This has some impact how users can use NEMO. More about that later.

The **make build** will do a basic install:

In [4]:
make build

(source nemo_start.sh ; make postconfig mknemos)
make[1]: Entering directory '/home/teuben/NEMO/demo'
Checking directory structure on Mon Jan 20 03:32:42 PM EST 2025
By teuben on pop-os
No tar symlink
Updating /home/teuben/NEMO/demo/adm/TIMESTAMP file on Mon Jan 20 03:32:42 PM EST 2025
Creating /home/teuben/NEMO/demo/etc/motd file on Mon Jan 20 03:32:42 PM EST 2025
Creating /home/teuben/NEMO/demo/nemo_local.csh file on Mon Jan 20 03:32:42 PM EST 2025
Creating /home/teuben/NEMO/demo/nemo_local.sh file on Mon Jan 20 03:32:42 PM EST 2025
Done with dirs at Mon Jan 20 03:32:42 PM EST 2025
cp config.h makedefs /home/teuben/NEMO/demo/lib
Installing scripts on Mon Jan 20 03:32:42 PM EST 2025
By teuben on pop-os
(cd /home/teuben/NEMO/demo/src/scripts; make install)
make[2]: Entering directory '/home/teuben/NEMO/demo/src/scripts'
cp Makefile.lib /home/teuben/NEMO/demo/lib
cp mknemo mkpdoc manlaser nemoman nemobench ldso nemo.version fitsedit fitsmerge nds9 shtool nemo.coverage showvar mir2nemo g

Check if the build was ok. A number of tests produce a data file which is regressed against a known answer. If these fail, you would see lines starting with **BSF** on the screen. 

The **make check** will do this:

In [5]:
make check


(source nemo_start.sh; src/scripts/testsuite | tee -a install.log )
TESTSUITE: OK   /home/teuben/NEMO/demo/src/image/fits
TESTSUITE: OK   /home/teuben/NEMO/demo/src/image/io
TESTSUITE: OK   /home/teuben/NEMO/demo/src/image/misc
TESTSUITE: OK   /home/teuben/NEMO/demo/src/image/rotcur
TESTSUITE: OK   /home/teuben/NEMO/demo/src/image/trans
TESTSUITE: OK   /home/teuben/NEMO/demo/src/image/wcs
TESTSUITE: OK   /home/teuben/NEMO/demo/src/kernel/fortran
TESTSUITE: OK   /home/teuben/NEMO/demo/src/kernel/io
TESTSUITE: OK   /home/teuben/NEMO/demo/src/kernel/misc
TESTSUITE: OK   /home/teuben/NEMO/demo/src/kernel/tab
TESTSUITE: OK   /home/teuben/NEMO/demo/src/nbody/cores
TESTSUITE: OK   /home/teuben/NEMO/demo/src/nbody/evolve/aarseth/nbody0
TESTSUITE: OK   /home/teuben/NEMO/demo/src/nbody/evolve/aarseth/tools
TESTSUITE: OK   /home/teuben/NEMO/demo/src/nbody/evolve/dehnen
TESTSUITE: OK   /home/teuben/NEMO/demo/src/nbody/evolve/directcode
TESTSUITE: OK   /home/teuben/NEMO/demo/src/nbody/evolve/flowco

And finally a benchmark to check how the speed compares with an otherwise random 2020-type laptop. Currently 6 nbody codes are run, each of them typically taking 5 seconds to compute.

The **make bench5** will do this:

In [6]:
make bench5


(source nemo_start.sh; /usr/bin/time src/scripts/nemo.bench mode=5 | tee -a install.log )
NEMOBENCH: (2024-10-18) bench764077 : nbody0=3072 nbody1=10240 nbody3=1000000 size=256 ften=1 clean=1 bsf=0 tmp=bench764077
pop-os
Linux pop-os 6.9.3-76060903-generic #202405300957~1726766035~22.04~4092a0e SMP PREEMPT_DYNAMIC Thu S x86_64 x86_64 x86_64 GNU/Linux
Mon Jan 20 03:34:15 PM EST 2025
Equal 5sec CPU time per code, no output files to check
Codes:  directcode [mkplummer] gyrfalcON hackcode1 [mkorbit] orbint potcode treecode1
Tstop: 4.28*1 5.64*1 7.89*1 787400.0*1 6.66*1 3.00*1 [5sec CPU for each on a 11th Gen Intel(R) Core(TM) i5-1135G7 up to 4.2GHz using gcc-11]
.1.23.456.
CPU_USAGE  directcode  :  4.72  4.71  0.00  0.00  0.00  721841408
CPU_USAGE  gyrfalcON   :  3.63  3.63  0.00  0.00  0.00  721841882
CPU_USAGE  hackcode1   :  4.05  4.04  0.00  0.00  0.00  721842246
CPU_USAGE  orbint      :  3.65  3.64  0.00  0.00  0.00  721842652
CPU_USAGE  potcode     :  3.67  3.66  0.00  0.00  0.00  72

Any score over 1000 is pretty good.

# Using NEMO
Now the package is ready for anybody to use. For this a user needs to source the **nemo_start.sh** file in their shell before NEMO commands are available. 

In our case here:


In [7]:
source $HOME/NEMO/demo/nemo_start.sh

and now the **nemo** command is available to check your active version of NEMO, and some other important
variables.

In [8]:
nemo

NEMO:        /home/teuben/NEMO/demo  - Version:4.5.1
YAPP:        /xs - default yapp plotting device
git:         Branch:master     Counter:12318      Date: 2025/01/20_10:43:06
python:      /home/teuben/NEMO/nemo/anaconda3/bin/python  - Python 3.12.4
OS_release:  Linux Description:	Pop!_OS 22.04 LTS


## A Plummer sphere

To create a Plummer (1911) sphere with 128 particles the **mkplummer** program can be used.

First a few ways to get help on this program


In [9]:
mkplummer help=h

out              : Output file name [???]
nbody            : Number of particles [???]
mlow             : Low mass fraction cutoff of Plummer dist [0]
mfrac            : Mass fraction used of Plummer distribution [0.999]
rfrac            : Radius fraction used of Plummer distribution
                              NOTE: the above two values are chosen so
                                    that m( rfrac ) = mfrac                 [22.8042468]
seed             : Seed for the random number generator [0]
time             : Time at which snapshot is taken [0.0]
zerocm           : Centrate snapshot (t/f)? [t]
scale            : Model scale factor (-1=virial 1=natural) [-1]
quiet            : 0=noisy 1=somewhat quiet 2=more quiet [0]
massname         : If used Mass-function name (e.g. n(m)) []
massexpr         : Mass function expression (e.g. pow(m,p)) [pow(m,p)]
masspars         : Mass function parameters (e.g. p,0.0) [p,0.0]
massrange        : Range for mass-spectrum (e.g. 1,2) [1,1]
headlin

Although the **man** command can be used to get the manual page displayed, NEMO programs also accept the **--man** argument to achieve the same thing:

In [10]:
mkplummer --man

MKPLUMMER(1NEMO)                                              MKPLUMMER(1NEMO)

NAME
       mkplummer - construct a Plummer model

SYNOPSIS
       mkplummer [parameters=values...]

DESCRIPTION
       mkplummer  constructs  an N-body realization of a Plummer model, with a
       spatial or mass based cut-off, after which it (optionally)  performs  a
       coordinate  transformation to the center-of-mass coordinate system. The
       data are then written into a file snap-file, in a standard N-body snap‐
       shot(5NEMO)  format.  The model is constructed in VIRIAL or NBODY units
       (M=G=-4E=1, with E the total energy), and finite spatial  extent  which
       can  be  regulated  by specifying mfrac or rfrac or using their default
       values.  The distribution function of a Plummer  model  is  spherically
       symmetric  and  isotropic, and is a polytrope of index n = 5.  See also
       Aarseth et al. (1974) and Plummer (1911).

       There is also an implementation in Dehn

Another frequently used reminder is the **--help** option, giving a shorthand of all keywords and their defaults. Keywords with the **???** value are required to get a value. In this case the first two arguments (out, nbody).

In [11]:
mkplummer --help


mkplummer out=??? nbody=??? mlow=0 mfrac=0.999 rfrac=22.8042468 seed=0 time=0.0 zerocm=t scale=-1 quiet=0 massname= massexpr=pow(m,p) masspars=p,0.0 massrange=1,1 headline= nmodel=1 mode=1 VERSION=3.0c


## Initial conditions

From the output of the **--help** the order of the keyword will matter if the order is kepts according to this list. So giving just two values binds them to the **out=** and **nbody=** resp.

In [12]:
mkplummer p128 128 seed=123

This this would have been equivalent:
```
     mkplummer out=p128 nbody=128
```



The contents of NEMO files is usually binary.  So viewing their contents in a more human friendly way is done with the program **tsf** (*type structured file*)

In [13]:
tsf p128

char Headline[28] "init_xrandom: seed used 123"
char History[41] "mkplummer p128 128 seed=123 VERSION=3.0c"
set SnapShot
  set Parameters
    int Nobj 128 
    double Time 0.00000 
  tes
  set Particles
    int CoordSystem 66306 
    double Mass[128] 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      . . .
    double PhaseSpace[128][2][3] -0.561583 -0.0653953 -0.541420 
      -0.106656 -0.119243 0.286915 4.85715 -0.329890 -1.61621 0.388497 
      0.207920 0.120547 0.593217 0.235838 -0.0272317 0.0567244 
      -0.0189743 -0.391682 0.425112 -0.0570265 -0.102289 -0.769960 
      . . .
  tes
tes


In [14]:
snapplot p128 yapp=installing1.png/png

![alt text](installing1.png "InitialConditions")


## Integrate it:

The **hackcode1** program is the original Barnes and Hut (1985) tree code N-logN algorithm to integrate an N-body system.

In [15]:
hackcode1 p128 p128.out tstop=2 


init_xrandom: seed used 123

       nbody        freq         eps         tol
         128       32.00      0.0500      1.0000

	options: mass,phase

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.000   -0.2396   -0.4900      6543        32        19      0.00

	        cm pos   -0.0000   -0.0000    0.0000
	        cm vel   -0.0000   -0.0000   -0.0000

	particle data written

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.031   -0.2397   -0.4885      6477        31        18      0.00

	        cm pos   -0.0000    0.0000   -0.0000
	        cm vel   -0.0000    0.0000   -0.0000

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.062   -0.2399   -0.4874      6456        31        19      0.00

	        cm pos   -0.0000    0.0000   -0.0000
	        cm vel    0.0000    0.0000    0.0001

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.094   -0.2403   -0.4864

Display some diagnostics, such as energy and center of mass motion

In [16]:
snapdiagplot p128.out yapp=installing2.png/png

### nemo Debug Info: 65 diagnostic frames read
Worst fractional energy loss dE/E = (E_t-E_0)/E_0 = 0.00695265 at T = 0.46875


This gives around 1% energy conservation, whichis not too bad, at least for galactic type simulations. For star clusters probably not too great at all.

![alt text](installing2.png "IntegrationDiagnostics")


Now plot in a two by two panel the X-Y projections of the integration at times 0, 1 and 2.

In [17]:
snapplot p128.out nxy=2,2 times=0,1,2 yapp=installing3.png/png

![alt text](installing3.png "EvolutionPlot")

## Compiling code

Although the build has done this, the **mknemo** command will recompile a program. This can be useful if the code needs to be edited for a bug fix and new feature. The **mknemo** command can be issued anywhere, and will show where the file in question lives. It can then easily be edited and recompiled, and added to $NEMOBIN. Except for editing, the **mknemo** command does all of this.

In [18]:
mknemo tsf

MKNEMO> Searching tsf.c: 
found one: /home/teuben/NEMO/demo/src/kernel/io/tsf.c
gcc -g -O2 -D_FILE_OFFSET_BITS=64 -D_LARGEFILE_SOURCE -D_LARGEFILE64_SOURCE  -fpic -rdynamic  -Dlinux -DSYSV  -I/home/teuben/NEMO/demo/inc -I/home/teuben/NEMO/demo/lib -I/home/teuben/NEMO/demo/opt/include    -Wall -Wimplicit-function-declaration  -D_GNU_SOURCE -std=c99  -o tsf tsf.c -L/home/teuben/NEMO/demo/lib -L/home/teuben/NEMO/demo/opt/lib          -lnemo -ldl  -lreadline -lhistory -lncurses  -lm   -L/usr/lib/gcc/x86_64-linux-gnu/11 -L/usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu -L/usr/lib/gcc/x86_64-linux-gnu/11/../../../../lib -L/lib/x86_64-linux-gnu -L/lib/../lib -L/usr/lib/x86_64-linux-gnu -L/usr/lib/../lib -L/usr/lib/gcc/x86_64-linux-gnu/11/../../.. -lgfortran -lm -lquadmath 



The output also reminds you wehre the **tsf.c** source code lives, so it can be easily put in the editor for some good hacking.


Here a program that uses graphics. These can be notoriously hard to link correctly.

In [19]:
mknemo tabplot

MKNEMO> Searching tabplot.c: 
found one: /home/teuben/NEMO/demo/src/kernel/tab/tabplot.c
gcc -g -O2 -D_FILE_OFFSET_BITS=64 -D_LARGEFILE_SOURCE -D_LARGEFILE64_SOURCE  -fpic -rdynamic  -Dlinux -DSYSV  -I/home/teuben/NEMO/demo/inc -I/home/teuben/NEMO/demo/lib -I/home/teuben/NEMO/demo/opt/include    -Wall -Wimplicit-function-declaration  -D_GNU_SOURCE -std=c99 -o tabplot tabplot.c \
	-L/home/teuben/NEMO/demo/lib -L/home/teuben/NEMO/demo/opt/lib          -lnemo -ldl  -lreadline -lhistory -lncurses  -lm  /home/teuben/NEMO/demo/lib/yapp_pgplot.o -L/usr/lib/pgplot5 -lcpgplot -lpgplot    -lSM -lICE -lXext -lX11   -L/usr/lib/gcc/x86_64-linux-gnu/11 -L/usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu -L/usr/lib/gcc/x86_64-linux-gnu/11/../../../../lib -L/lib/x86_64-linux-gnu -L/lib/../lib -L/usr/lib/x86_64-linux-gnu -L/usr/lib/../lib -L/usr/lib/gcc/x86_64-linux-gnu/11/../../.. -lgfortran -lm -lquadmath  -lm



and a simple benchmark if graphics works in the **tabgen + tabplot** pipe:

In [20]:
tabgen - nr=10 nc=4 | tabplot - ycol=2,3,4 point=2,0.1,3,0.2,4,0.3 color=2,3,4 yapp=installing4.png/png
pwd
ls -l installing4.png

### nemo Debug Info: read 10 points
### nemo Debug Info: min and max value in xcolumns 1: [0.139570 : 0.802354]
### nemo Debug Info: min and max value in ycolumns 2,3,4: [0.004308 : 0.988963]
### nemo Debug Info: X:min and max value reset to : [0.106431 : 0.835493]
### nemo Debug Info: Y:min and max value reset to : [-0.044925 : 1.038196]
/home/teuben/NEMO/demo
-rw-rw-r-- 1 teuben teuben 4769 Jan 20 15:34 installing4.png


![alt text](installing4.png "ScatterPlot")

In [21]:
date1=$(date)
echo "Started at:   $date0"
echo "Done at:      $date1"
(cd $NEMOBIN; ls | wc -l)

Started at:   Mon Jan 20 03:32:36 PM EST 2025
Done at:      Mon Jan 20 03:34:41 PM EST 2025
300


How did you do?  Did you get 300 executables in $NEMOBIN, did your install take about 2 minutes?